<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/mistral_aoc_demo_june2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai -q
!pip install colab-env -q

In [2]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    # !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for AOCC outputs
class FlightOperationStatus(BaseModel):
    flight_id: str
    status: str # e.g., "On Time", "Delayed", "Cancelled"
    gate: str
    runway_status: str
    delay_reason: str = None
    estimated_new_time: str = None
    flagged_issues: dict = None

class AircraftMaintenanceReport(BaseModel):
    aircraft_tail_number: str
    maintenance_status: str # e.g., "Scheduled", "In Progress", "Completed", "AOG"
    last_service_date: str
    next_service_due: str
    issues_reported: list
    parts_needed: list

print("Creating AI agents for Airline Operation Control Center Domain...")

# Agent Definitions (Airline Operation Control Center Domain)

# 1. Flight Planning Agent
flight_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for planning flight routes, optimizing fuel, and managing flight schedules.",
    name="flight-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "design_flight_plan",
                "description": "Design a flight plan based on origin, destination, and optional preferred route.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure airport IATA code."},
                        "destination": {"type": "string", "description": "Arrival airport IATA code."},
                        "preferred_route": {"type": "string", "description": "Optional: Specific route preference (e.g., 'direct', 'ETOPS')."}
                    },
                    "required": ["origin", "destination"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_flight_schedule",
                "description": "Log the scheduled departure and arrival times for a flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Unique identifier for the flight."},
                        "scheduled_departure": {"type": "string", "description": "Scheduled departure timestamp (ISO format)."},
                        "scheduled_arrival": {"type": "string", "description": "Scheduled arrival timestamp (ISO format)."}
                    },
                    "required": ["flight_id", "scheduled_departure", "scheduled_arrival"]
                }
            }
        }
    ]
)
print(f"Flight Planning Agent '{flight_planning_agent.name}' created with ID: {flight_planning_agent.id}")

# 2. Air Traffic Control Liaison Agent
air_traffic_control_liaison_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for communicating with air traffic control, predicting airspace congestion, and analyzing slot availability.",
    name="atc-liaison-agent",
    tools=[
        {"type": "web_search"}, # For searching external databases for real-time NOTAMs or weather.
        {
            "type": "function",
            "function": {
                "name": "predict_airspace_congestion",
                "description": "Predict the level of airspace congestion for a given airport and time window.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "airport_code": {"type": "string", "description": "IATA code of the airport."},
                        "time_window": {"type": "string", "description": "Time period for congestion prediction (e.g., 'next 4 hours')."}
                    },
                    "required": ["airport_code", "time_window"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "check_slot_availability",
                "description": "Check for available takeoff/landing slots at a specific airport.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "airport_code": {"type": "string", "description": "IATA code of the airport."},
                        "requested_time": {"type": "string", "description": "Desired time for the slot (ISO format)."}
                    },
                    "required": ["airport_code", "requested_time"]
                }
            }
        }
    ]
)
print(f"Air Traffic Control Liaison Agent '{air_traffic_control_liaison_agent.name}' created with ID: {air_traffic_control_liaison_agent.id}")

# 3. Disruption Management Agent
disruption_management_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="disruption-management-agent",
    description="Agent to predict and manage operational disruptions (e.g., delays, cancellations) and suggest recovery plans.",
    instructions="Predict disruption impact and propose effective recovery strategies.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "flight_operation_status",
                "schema": FlightOperationStatus.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "predict_disruption_impact",
                "description": "Predict the impact level of a disruption (e.g., 'weather', 'ATC delay') on flights.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the affected flight."},
                        "disruption_type": {"type": "string", "description": "The type of disruption (e.g., 'weather', 'technical issue')."}
                    },
                    "required": ["flight_id", "disruption_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "propose_recovery_plan",
                "description": "Suggest a recovery plan for a disrupted flight or set of flights.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight requiring recovery."},
                        "disruption_details": {"type": "string", "description": "Details of the disruption (e.g., 'runway closed for 2 hours')."}
                    },
                    "required": ["flight_id", "disruption_details"]
                }
            }
        }
    ]
)
print(f"Disruption Management Agent '{disruption_management_agent.name}' created with ID: {disruption_management_agent.id}")

# 4. Crew Scheduling Agent
crew_scheduling_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for optimizing crew assignments, managing duty limits, and simulating roster changes.",
    name="crew-scheduling-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "assign_crew_to_flight",
                "description": "Assign pilots and cabin crew to a specific flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight."},
                        "pilot_ids": {"type": "array", "items": {"type": "string"}, "description": "List of pilot IDs."},
                        "cabin_crew_ids": {"type": "array", "items": {"type": "string"}, "description": "List of cabin crew IDs."}
                    },
                    "required": ["flight_id", "pilot_ids", "cabin_crew_ids"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "simulate_roster_change",
                "description": "Simulate the impact of a roster change on crew legality and availability.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "crew_member_id": {"type": "string", "description": "Identifier of the crew member."},
                        "new_assignment": {"type": "string", "description": "Description of the new assignment (e.g., 'Flight AC456 on 2025-07-01')."}
                    },
                    "required": ["crew_member_id", "new_assignment"]
                }
            }
        }
    ]
)
print(f"Crew Scheduling Agent '{crew_scheduling_agent.name}' created with ID: {crew_scheduling_agent.id}")


# 5. Aircraft Maintenance Agent
aircraft_maintenance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing aircraft maintenance schedules, tracking part availability, and logging service records.",
    name="aircraft-maintenance-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "schedule_maintenance",
                "description": "Schedule a maintenance event for a specific aircraft.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aircraft_id": {"type": "string", "description": "Tail number of the aircraft."},
                        "maintenance_type": {"type": "string", "description": "Type of maintenance (e.g., 'A-check', 'engine repair')."},
                        "preferred_date": {"type": "string", "description": "Preferred date for maintenance (ISO format)."}
                    },
                    "required": ["aircraft_id", "maintenance_type", "preferred_date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "check_parts_inventory",
                "description": "Check the availability and location of a specific aircraft part in inventory.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "part_number": {"type": "string", "description": "Identifier of the aircraft part."}
                    },
                    "required": ["part_number"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_maintenance_record",
                "description": "Log the completion of a maintenance task for an aircraft.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aircraft_id": {"type": "string", "description": "Tail number of the aircraft."},
                        "service_description": {"type": "string", "description": "Description of the maintenance performed."},
                        "date_completed": {"type": "string", "description": "Date of completion (ISO format)."}
                    },
                    "required": ["aircraft_id", "service_description", "date_completed"]
                }
            }
        }
    ]
)
print(f"Aircraft Maintenance Agent '{aircraft_maintenance_agent.name}' created with ID: {aircraft_maintenance_agent.id}")

# 6. Operational Data Analysis Agent
operational_data_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for accessing, analyzing, and interpreting flight operational data.",
    name="operational-data-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_flight_data",
                "description": "Retrieve raw or processed operational data from a specific flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Unique identifier for the flight."},
                        "data_type": {"type": "string", "description": "Optional: Type of data to retrieve (e.g., 'telemetry', 'fuel burn')."}
                    },
                    "required": ["flight_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_operational_efficiency",
                "description": "Perform statistical analysis on operational data to determine efficiency metrics.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_json": {"type": "string", "description": "JSON string of numerical operational data for analysis."},
                        "comparison_data_json": {"type": "string", "description": "Optional: JSON string of comparison data."}
                    },
                    "required": ["data_json"]
                }
            }
        }
    ]
)
print(f"Operational Data Analysis Agent '{operational_data_analysis_agent.name}' created with ID: {operational_data_analysis_agent.id}")

# 7. Compliance & Safety Agent
compliance_safety_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for ensuring adherence to aviation regulations and safety protocols.",
    name="compliance-safety-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_aviation_regulation",
                "description": "Check a specific aspect of aviation operations against relevant regulatory guidelines.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aspect": {"type": "string", "description": "Aspect to check (e.g., 'flight duty limitations', 'aircraft certification')."},
                        "region": {"type": "string", "description": "Regulatory region (e.g., 'FAA', 'EASA', 'TC')."}
                    },
                    "required": ["aspect", "region"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "submit_safety_report",
                "description": "Simulate submission of a safety report for an incident or observation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "incident_type": {"type": "string", "description": "Type of incident (e.g., 'bird strike', 'runway excursion')."},
                        "summary": {"type": "string", "description": "Summary of the incident or observation."}
                    },
                    "required": ["incident_type", "summary"]
                }
            }
        }
    ]
)
print(f"Compliance & Safety Agent '{compliance_safety_agent.name}' created with ID: {compliance_safety_agent.id}")

# 8. Incident Response Agent
incident_response_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for analyzing operational incidents, identifying root causes, and predicting cascading effects.",
    name="incident-response-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "analyze_incident",
                "description": "Analyze existing operational data to infer or confirm the root cause of an incident.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "incident_id": {"type": "string", "description": "Identifier of the incident."},
                        "operational_data_json": {"type": "string", "description": "JSON string of relevant operational data for analysis."}
                    },
                    "required": ["incident_id", "operational_data_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "predict_cascading_effects",
                "description": "Predict potential cascading effects of an incident on other flights or operations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "incident_id": {"type": "string", "description": "Identifier of the incident."},
                        "affected_flights": {"type": "array", "items": {"type": "string"}, "description": "List of flight IDs potentially affected."}
                    },
                    "required": ["incident_id", "affected_flights"]
                }
            }
        }
    ]
)
print(f"Incident Response Agent '{incident_response_agent.name}' created with ID: {incident_response_agent.id}")

# 9. Ground Operations Coordination Agent
ground_operations_coordination_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for coordinating with ground staff, baggage handling, and catering services.",
    name="ground-ops-coordination-agent",
    tools=[
        {"type": "web_search"}, # For looking up external ground service provider contact info or service status.
        {
            "type": "function",
            "function": {
                "name": "track_ground_activity",
                "description": "Track the completion status of a specific ground activity for a flight (e.g., 'baggage loading', 'refueling').",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight."},
                        "activity_name": {"type": "string", "description": "Name of the ground activity to track."}
                    },
                    "required": ["flight_id", "activity_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "request_ground_service",
                "description": "Request a specific ground service for a flight (e.g., 'catering delivery', 'pushback').",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight."},
                        "service_description": {"type": "string", "description": "Detailed description of the service requested."}
                    },
                    "required": ["flight_id", "service_description"]
                }
            }
        }
    ]
)
print(f"Ground Operations Coordination Agent '{ground_operations_coordination_agent.name}' created with ID: {ground_operations_coordination_agent.id}")


# NEW AGENTS ADDED
# 10. Passenger Coordinator Agent
passenger_coordinator_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing passenger information, rebooking affected passengers during disruptions, and communicating updates.",
    name="passenger-coordinator-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_passenger_manifest",
                "description": "Retrieves the passenger list and their details for a given flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight."}
                    },
                    "required": ["flight_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "rebook_passenger",
                "description": "Rebooks a passenger from a current flight to an alternative flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "passenger_id": {"type": "string", "description": "Unique identifier for the passenger."},
                        "current_flight_id": {"type": "string", "description": "Identifier of the passenger's current flight."},
                        "new_flight_id": {"type": "string", "description": "Identifier of the new flight to rebook to."}
                    },
                    "required": ["passenger_id", "current_flight_id", "new_flight_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "send_passenger_notification",
                "description": "Sends an update or notification to a specific passenger.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "passenger_id": {"type": "string", "description": "Unique identifier for the passenger."},
                        "message_content": {"type": "string", "description": "The content of the message to send."},
                        "contact_method": {"type": "string", "description": "Method of contact (e.g., 'email', 'SMS')."}
                    },
                    "required": ["passenger_id", "message_content", "contact_method"]
                }
            }
        }
    ]
)
print(f"Passenger Coordinator Agent '{passenger_coordinator_agent.name}' created with ID: {passenger_coordinator_agent.id}")

# 11. Crew Coordinator Agent
crew_coordinator_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing crew assignments, ensuring crew legality (duty time limits), and coordinating crew logistics.",
    name="crew-coordinator-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_crew_legality",
                "description": "Checks if a crew member is compliant with aviation duty time regulations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "crew_member_id": {"type": "string", "description": "Identifier of the crew member."}
                    },
                    "required": ["crew_member_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "find_available_crew",
                "description": "Searches for available crew members with specific roles and qualifications.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "role": {"type": "string", "description": "Role of the crew member (e.g., 'Pilot', 'Flight Attendant')."},
                        "required_qualifications": {"type": "array", "items": {"type": "string"}, "description": "List of required qualifications (e.g., 'Boeing 737 rated')."}
                    },
                    "required": ["role", "required_qualifications"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assign_crew_to_duty",
                "description": "Assigns a crew member to a specific duty or flight, updating their schedule.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "crew_member_id": {"type": "string", "description": "Identifier of the crew member."},
                        "duty_details": {"type": "string", "description": "Description of the duty or flight assignment."}
                    },
                    "required": ["crew_member_id", "duty_details"]
                }
            }
        }
    ]
)
print(f"Crew Coordinator Agent '{crew_coordinator_agent.name}' created with ID: {crew_coordinator_agent.id}")

# 12. Flight Dispatcher Agent
flight_dispatcher_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for real-time flight monitoring, weather analysis, fuel load optimization, and issuing flight releases.",
    name="flight-dispatcher-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "monitor_flight_progress",
                "description": "Monitors the current position, speed, altitude, and status of an airborne flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight to monitor."}
                    },
                    "required": ["flight_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_weather_impact",
                "description": "Analyzes weather conditions along a flight route and assesses potential impact on operations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "route": {"type": "string", "description": "Description of the flight route (e.g., 'Montreal to Los Angeles')."},
                        "time_window": {"type": "string", "description": "Time period for weather analysis (e.g., 'next 6 hours')."}
                    },
                    "required": ["route", "time_window"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "calculate_fuel_requirements",
                "description": "Calculates the optimal fuel load required for a flight based on route, payload, and contingency.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight."},
                        "route": {"type": "string", "description": "Description of the flight route."},
                        "payload_weight": {"type": "number", "description": "Total payload weight in kg."},
                        "contingency_fuel_percent": {"type": "number", "description": "Percentage of extra fuel for contingency (e.g., 5 for 5%)."}
                    },
                    "required": ["flight_id", "route", "payload_weight", "contingency_fuel_percent"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "issue_flight_release",
                "description": "Formally issues a flight release, authorizing a flight for departure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_id": {"type": "string", "description": "Identifier of the flight to release."},
                        "conditions_summary": {"type": "string", "description": "Summary of current operational conditions for release."}
                    },
                    "required": ["flight_id", "conditions_summary"]
                }
            }
        }
    ]
)
print(f"Flight Dispatcher Agent '{flight_dispatcher_agent.name}' created with ID: {flight_dispatcher_agent.id}")

# 13. Budget Finance Agent
budget_finance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for monitoring operational budgets, tracking expenses, forecasting costs, and optimizing financial efficiency within airline operations.",
    name="budget-finance-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_operational_budget_status",
                "description": "Retrieves the current status of a specific operational budget for a given department and time period.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "department": {"type": "string", "description": "The operational department (e.g., 'Flight Operations', 'Maintenance')."},
                        "period": {"type": "string", "description": "The financial period (e.g., 'Q3 2025', 'FY 2025')."}
                    },
                    "required": ["department", "period"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_operational_expense",
                "description": "Logs an operational expense against a specific category, optionally linked to a flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expense_category": {"type": "string", "description": "The category of the expense (e.g., 'fuel', 'landing fees', 'crew salaries')."},
                        "amount": {"type": "number", "description": "The amount of the expense."},
                        "flight_id": {"type": "string", "description": "Optional: Identifier of the flight if the expense is flight-specific."}
                    },
                    "required": ["expense_category", "amount"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "forecast_operational_costs",
                "description": "Forecasts future operational costs for a given cost type and time horizon.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cost_type": {"type": "string", "description": "The type of cost to forecast (e.g., 'fuel', 'maintenance', 'personnel')."},
                        "time_horizon": {"type": "string", "description": "The period for the forecast (e.g., 'next month', 'next quarter')."}
                    },
                    "required": ["cost_type", "time_horizon"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_cost_efficiency",
                "description": "Analyzes operational cost data to identify areas for efficiency improvement.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_json": {"type": "string", "description": "JSON string of numerical cost data for analysis (e.g., [{'item': 'fuel', 'cost': 1000}, {'item': 'fees', 'cost': 200}])."},
                        "benchmark_data_json": {"type": "string", "description": "Optional: JSON string of benchmark data for comparison."}
                    },
                    "required": ["data_json"]
                }
            }
        }
    ]
)
print(f"Budget Finance Agent '{budget_finance_agent.name}' created with ID: {budget_finance_agent.id}")


print("\nAll new Airline Operation Control Center agents have been defined.")

# Mock functions for AOCC domain tools
# These mock functions provide simulated responses for the new agents' tools.

# Flight Planning Agent Mocks
def design_flight_plan(origin: str, destination: str, preferred_route: str = None):
    """MOCK function to design a flight plan."""
    print(f"\n[DEBUG] MOCK CALL: design_flight_plan Origin: '{origin}', Destination: '{destination}', Route: '{preferred_route}'")
    if origin == "YUL" and destination == "LAX":
        return {"status": "success", "flight_id": "AC123", "route": "Great Circle", "message": "Flight plan optimized for efficiency from Montreal to Los Angeles."}
    return {"status": "success", "flight_id": "FL-001", "route": "Standard", "message": "Basic flight plan designed."}

def log_flight_schedule(flight_id: str, scheduled_departure: str, scheduled_arrival: str):
    """MOCK function to log flight schedule."""
    print(f"\n[DEBUG] MOCK CALL: log_flight_schedule Flight: {flight_id}, Departure: {scheduled_departure}, Arrival: {scheduled_arrival}")
    return {"status": "logged", "flight_id": flight_id, "message": "Flight schedule logged successfully."}

# Air Traffic Control Liaison Agent Mocks
def predict_airspace_congestion(airport_code: str, time_window: str):
    """MOCK function to predict airspace congestion."""
    print(f"\n[DEBUG] MOCK CALL: predict_airspace_congestion Airport: {airport_code}, Time Window: {time_window}")
    if airport_code == "LAX" and "next 4 hours" in time_window:
        return {"status": "warning", "congestion_level": "High", "message": "High congestion predicted at LAX."}
    return {"status": "low", "congestion_level": "Low", "message": "Low congestion expected."}

def check_slot_availability(airport_code: str, requested_time: str):
    """MOCK function to check slot availability."""
    print(f"\n[DEBUG] MOCK CALL: check_slot_availability Airport: {airport_code}, Time: {requested_time}")
    if airport_code == "YUL" and "10:00" in requested_time:
        return {"status": "available", "slot_time": "2025-06-25T10:00:00Z", "message": "Slot available at YUL."}
    return {"status": "unavailable", "message": "No slot available at requested time."}

# Disruption Management Agent Mocks
def predict_disruption_impact(flight_id: str, disruption_type: str):
    """MOCK function to predict disruption impact."""
    print(f"\n[DEBUG] MOCK CALL: predict_disruption_impact Flight: {flight_id}, Disruption: {disruption_type}")
    if disruption_type == "weather" and flight_id == "AC123":
        return {"status": "warning", "impact_level": "High", "affected_flights": ["AC123", "UA456"], "message": "Severe weather predicted, expect delays."}
    return {"status": "low_impact", "message": "Minor disruption, manageable."}

def propose_recovery_plan(flight_id: str, disruption_details: str):
    """MOCK function to propose recovery plan."""
    print(f"\n[DEBUG] MOCK CALL: propose_recovery_plan Flight: {flight_id}, Details: '{disruption_details}'")
    if "runway closed" in disruption_details:
        return {"status": "plan_proposed", "plan": "Divert to alternative airport, rebook passengers.", "message": "Recovery plan drafted."}
    return {"status": "plan_proposed", "plan": "Adjust schedule slightly.", "message": "Minor recovery plan."}

# Crew Scheduling Agent Mocks
def assign_crew_to_flight(flight_id: str, pilot_ids: list, cabin_crew_ids: list):
    """MOCK function to assign crew to flight."""
    print(f"\n[DEBUG] MOCK CALL: assign_crew_to_flight Flight: {flight_id}, Pilots: {pilot_ids}, Cabin Crew: {cabin_crew_ids}")
    return {"status": "assigned", "flight_id": flight_id, "message": "Crew assigned successfully."}

def simulate_roster_change(crew_member_id: str, new_assignment: str):
    """MOCK function to simulate roster change."""
    print(f"\n[DEBUG] MOCK CALL: simulate_roster_change Crew: {crew_member_id}, New Assignment: '{new_assignment}'")
    if crew_member_id == "PILOT-001" and "AC456" in new_assignment:
        return {"status": "simulation_complete", "legality_check": "compliant", "message": "Roster change simulated, compliant."}
    return {"status": "simulation_complete", "legality_check": "review_needed", "message": "Roster change simulated, review needed."}

# Aircraft Maintenance Agent Mocks
def schedule_maintenance(aircraft_id: str, maintenance_type: str, preferred_date: str):
    """MOCK function to schedule maintenance."""
    print(f"\n[DEBUG] MOCK CALL: schedule_maintenance Aircraft: {aircraft_id}, Type: {maintenance_type}, Date: {preferred_date}")
    return {"status": "scheduled", "aircraft_id": aircraft_id, "message": "Maintenance scheduled."}

def check_parts_inventory(part_number: str):
    """MOCK function to check parts inventory."""
    print(f"\n[DEBUG] MOCK CALL: check_parts_inventory Part: {part_number}")
    if part_number == "ENG-001":
        return {"status": "in_stock", "quantity": 5, "location": "Warehouse A", "message": "Part in stock."}
    return {"status": "out_of_stock", "message": "Part not in stock."}

def log_maintenance_record(aircraft_id: str, service_description: str, date_completed: str):
    """MOCK function to log maintenance record."""
    print(f"\n[DEBUG] MOCK CALL: log_maintenance_record Aircraft: {aircraft_id}, Service: '{service_description}', Date: {date_completed}")
    return {"status": "logged", "aircraft_id": aircraft_id, "message": "Maintenance record logged."}

# Operational Data Analysis Agent Mocks
def get_flight_data(flight_id: str, data_type: str = "telemetry"):
    """MOCK function to retrieve flight data."""
    print(f"\n[DEBUG] MOCK CALL: get_flight_data Flight: {flight_id}, Type: {data_type}")
    if flight_id == "AC123" and data_type == "telemetry":
        return {"status": "success", "data": "Telemetry data for AC123: [speed: 500kts, alt: 35000ft]", "message": "Flight data retrieved."}
    return {"status": "not_found", "message": "Flight data not found."}

def analyze_operational_efficiency(data_json: str, comparison_data_json: str = None):
    """MOCK function to analyze operational efficiency."""
    print(f"\n[DEBUG] MOCK CALL: analyze_operational_efficiency Data: {data_json}, Comparison: {comparison_data_json}")
    data = json.loads(data_json)
    avg_speed = sum(data) / len(data) if data else 0
    return {"status": "success", "average_speed": avg_speed, "message": "Efficiency analysis complete."}

# Compliance & Safety Agent Mocks
def check_aviation_regulation(aspect: str, region: str):
    """MOCK function to check aviation regulation."""
    print(f"\n[DEBUG] MOCK CALL: check_aviation_regulation Aspect: {aspect}, Region: {region}")
    if aspect == "flight duty limitations" and region == "EASA":
        return {"status": "compliant", "details": "Compliant with EASA FTL regulations.", "message": "Regulation check complete."}
    return {"status": "non_compliant", "details": "Requires further review.", "message": "Potential non-compliance."}

def submit_safety_report(incident_type: str, summary: str):
    """MOCK function to submit safety report."""
    print(f"\n[DEBUG] MOCK CALL: submit_safety_report Type: {incident_type}, Summary: '{summary}'")
    return {"status": "submitted", "report_id": "SR-001", "message": "Safety report submitted."}

# Incident Response Agent Mocks
def analyze_incident(incident_id: str, operational_data_json: str):
    """MOCK function to analyze incident."""
    print(f"\n[DEBUG] MOCK CALL: analyze_incident Incident: {incident_id}, Data: {operational_data_json}")
    if incident_id == "INC-001" and "engine anomaly" in operational_data_json:
        return {"status": "success", "root_cause": "Minor engine malfunction.", "confidence": 0.9, "message": "Incident analysis complete."}
    return {"status": "needs_more_data", "message": "Further data needed for root cause analysis."}

def predict_cascading_effects(incident_id: str, affected_flights: list):
    """MOCK function to predict cascading effects."""
    print(f"\n[DEBUG] MOCK CALL: predict_cascading_effects Incident: {incident_id}, Affected Flights: {affected_flights}")
    if incident_id == "INC-001" and "AC123" in affected_flights:
        return {"status": "warning", "cascading_impact": "Potential delays for connecting flights, crew rest issues.", "message": "Cascading effects predicted."}
    return {"status": "no_major_impact", "message": "No significant cascading effects predicted."}

# Ground Operations Coordination Agent Mocks
def track_ground_activity(flight_id: str, activity_name: str):
    """MOCK function to track ground activity."""
    print(f"\n[DEBUG] MOCK CALL: track_ground_activity Flight: {flight_id}, Activity: '{activity_name}'")
    if flight_id == "AC123" and activity_name == "baggage loading":
        return {"status": "completed", "completion_time": "2025-06-25T09:30:00Z", "message": "Baggage loading complete."}
    return {"status": "in_progress", "message": "Activity ongoing or not found."}

def request_ground_service(flight_id: str, service_description: str):
    """MOCK function to request ground service."""
    print(f"\n[DEBUG] MOCK CALL: request_ground_service Flight: {flight_id}, Service: '{service_description}'")
    return {"status": "requested", "service_id": "GND-001", "message": "Ground service requested."}

# Passenger Coordinator Agent Mocks
def get_passenger_manifest(flight_id: str):
    """MOCK function to retrieve passenger manifest."""
    print(f"\n[DEBUG] MOCK CALL: get_passenger_manifest Flight: {flight_id}")
    if flight_id == "AC123":
        return {"status": "success", "flight_id": flight_id, "passengers": [{"id": "P001", "name": "Alice Smith", "email": "alice@example.com"}, {"id": "P002", "name": "Bob Johnson", "email": "bob@example.com"}], "message": "Manifest retrieved."}
    return {"status": "not_found", "message": "Flight or manifest not found."}

def rebook_passenger(passenger_id: str, current_flight_id: str, new_flight_id: str):
    """MOCK function to rebook a passenger."""
    print(f"\n[DEBUG] MOCK CALL: rebook_passenger Passenger: {passenger_id}, Old Flight: {current_flight_id}, New Flight: {new_flight_id}")
    return {"status": "rebooked", "passenger_id": passenger_id, "new_flight_id": new_flight_id, "message": f"Passenger {passenger_id} rebooked to {new_flight_id}."}

def send_passenger_notification(passenger_id: str, message_content: str, contact_method: str):
    """MOCK function to send passenger notification."""
    print(f"\n[DEBUG] MOCK CALL: send_passenger_notification Passenger: {passenger_id}, Content: '{message_content}', Method: {contact_method}")
    return {"status": "sent", "passenger_id": passenger_id, "message": "Notification sent successfully."}

# Crew Coordinator Agent Mocks
def check_crew_legality(crew_member_id: str):
    """MOCK function to check crew legality."""
    print(f"\n[DEBUG] MOCK CALL: check_crew_legality Crew: {crew_member_id}")
    if crew_member_id == "PILOT-001":
        return {"status": "compliant", "message": "Crew member is legal for duty."}
    return {"status": "non_compliant", "message": "Crew member exceeds duty limits or requires rest."}

def find_available_crew(role: str, required_qualifications: list):
    """MOCK function to find available crew."""
    print(f"\n[DEBUG] MOCK CALL: find_available_crew Role: {role}, Qualifications: {required_qualifications}")
    if role == "Pilot" and "Boeing 737" in required_qualifications:
        return {"status": "success", "crew_found": ["PILOT-002", "PILOT-003"], "message": "Available qualified pilots found."}
    return {"status": "not_found", "message": "No matching crew found."}

def assign_crew_to_duty(crew_member_id: str, duty_details: str):
    """MOCK function to assign crew to duty."""
    print(f"\n[DEBUG] MOCK CALL: assign_crew_to_duty Crew: {crew_member_id}, Duty: '{duty_details}'")
    return {"status": "assigned", "crew_member_id": crew_member_id, "duty_details": duty_details, "message": "Crew member assigned."}

# Flight Dispatcher Agent Mocks
def monitor_flight_progress(flight_id: str):
    """MOCK function to monitor flight progress."""
    print(f"\n[DEBUG] MOCK CALL: monitor_flight_progress Flight: {flight_id}")
    if flight_id == "AC123":
        return {"status": "in_flight", "current_position": "N45.50, W73.57", "altitude": "35000 ft", "speed": "500 knots", "message": "Flight AC123 currently en route."}
    return {"status": "unknown", "message": "Flight not found or not in progress."}

def analyze_weather_impact(route: str, time_window: str):
    """MOCK function to analyze weather impact."""
    print(f"\n[DEBUG] MOCK CALL: analyze_weather_impact Route: '{route}', Time Window: {time_window}")
    if "LAX" in route and "severe thunderstorms" in time_window:
        return {"status": "warning", "impact": "Potential diversions or delays", "details": "Thunderstorms expected near destination.", "message": "Weather impact analyzed."}
    return {"status": "clear", "impact": "None", "message": "No significant weather impact."}

def calculate_fuel_requirements(flight_id: str, route: str, payload_weight: float, contingency_fuel_percent: float):
    """MOCK function to calculate fuel requirements."""
    print(f"\n[DEBUG] MOCK CALL: calculate_fuel_requirements Flight: {flight_id}, Route: '{route}', Payload: {payload_weight}kg, Contingency: {contingency_fuel_percent}%")
    return {"status": "success", "required_fuel_kg": 25000, "message": "Fuel calculation complete."}

def issue_flight_release(flight_id: str, conditions_summary: str):
    """MOCK function to issue flight release."""
    print(f"\n[DEBUG] MOCK CALL: issue_flight_release Flight: {flight_id}, Conditions: '{conditions_summary}'")
    return {"status": "released", "flight_id": flight_id, "release_timestamp": "2025-06-25T10:00:00Z", "message": "Flight released for departure."}

# Budget Finance Agent Mocks
def get_operational_budget_status(department: str, period: str):
    """MOCK function to retrieve operational budget status."""
    print(f"\n[DEBUG] MOCK CALL: get_operational_budget_status Department: {department}, Period: {period}")
    if department == "Flight Operations" and period == "Q3 2025":
        return {"status": "success", "department": department, "period": period, "allocated_budget": 5000000.00, "spent_to_date": 3500000.00, "remaining_budget": 1500000.00, "message": "Budget status retrieved."}
    return {"status": "not_found", "message": "Budget status not found for the specified department and period."}

def log_operational_expense(expense_category: str, amount: float, flight_id: str = None):
    """MOCK function to log an operational expense."""
    print(f"\n[DEBUG] MOCK CALL: log_operational_expense Category: {expense_category}, Amount: {amount}, Flight: {flight_id}")
    return {"status": "logged", "expense_id": f"EXP-{int(time.time())}", "category": expense_category, "amount": amount, "flight_id": flight_id, "message": "Expense logged successfully."}

def forecast_operational_costs(cost_type: str, time_horizon: str):
    """MOCK function to forecast operational costs."""
    print(f"\n[DEBUG] MOCK CALL: forecast_operational_costs Cost Type: {cost_type}, Time Horizon: {time_horizon}")
    if cost_type == "fuel" and time_horizon == "next month":
        return {"status": "success", "cost_type": cost_type, "forecasted_amount": 12000000.00, "unit": "USD", "message": "Fuel cost forecast generated."}
    return {"status": "success", "cost_type": cost_type, "forecasted_amount": 0.0, "message": "Forecast generated (placeholder)."}

def analyze_cost_efficiency(data_json: str, benchmark_data_json: str = None):
    """MOCK function to analyze cost efficiency."""
    print(f"\n[DEBUG] MOCK CALL: analyze_cost_efficiency Data: {data_json}, Benchmark: {benchmark_data_json}")
    try:
        data = json.loads(data_json)
        total_cost = sum(item.get("cost", 0) for item in data)
        efficiency_metric = total_cost / len(data) if data else 0
        return {"status": "success", "total_cost": total_cost, "efficiency_metric": efficiency_metric, "message": "Cost efficiency analysis complete."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for data."}


# Master Tool Executor Mapping (Updated for AOCC Domain)
tool_executor = {
    "design_flight_plan": design_flight_plan,
    "log_flight_schedule": log_flight_schedule,
    "predict_airspace_congestion": predict_airspace_congestion,
    "check_slot_availability": check_slot_availability,
    "predict_disruption_impact": predict_disruption_impact,
    "propose_recovery_plan": propose_recovery_plan,
    "assign_crew_to_flight": assign_crew_to_flight,
    "simulate_roster_change": simulate_roster_change,
    "schedule_maintenance": schedule_maintenance,
    "check_parts_inventory": check_parts_inventory,
    "log_maintenance_record": log_maintenance_record,
    "get_flight_data": get_flight_data,
    "analyze_operational_efficiency": analyze_operational_efficiency,
    "check_aviation_regulation": check_aviation_regulation,
    "submit_safety_report": submit_safety_report,
    "analyze_incident": analyze_incident,
    "predict_cascading_effects": predict_cascading_effects,
    "track_ground_activity": track_ground_activity,
    "request_ground_service": request_ground_service,
    # New agent tools
    "get_passenger_manifest": get_passenger_manifest,
    "rebook_passenger": rebook_passenger,
    "send_passenger_notification": send_passenger_notification,
    "check_crew_legality": check_crew_legality,
    "find_available_crew": find_available_crew,
    "assign_crew_to_duty": assign_crew_to_duty,
    "monitor_flight_progress": monitor_flight_progress,
    "analyze_weather_impact": analyze_weather_impact,
    "calculate_fuel_requirements": calculate_fuel_requirements,
    "issue_flight_release": issue_flight_release,
    "get_operational_budget_status": get_operational_budget_status,
    "log_operational_expense": log_operational_expense,
    "forecast_operational_costs": forecast_operational_costs,
    "analyze_cost_efficiency": analyze_cost_efficiency,
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: General aviation information retrieved."
}

# Function to standardize tools for client.chat.complete (remains the same)
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# Test Case Execution for AOCC Agents
print("\n--- Executing Test Cases for Airline Operation Control Center Agents (via chat completions) ---")

test_cases = [
    {
        "agent": flight_planning_agent,
        "name": "Flight Planning Agent",
        "query": "Design a flight from Montreal to Los Angeles.",
        "expected_tool_call": "design_flight_plan"
    },
    {
        "agent": air_traffic_control_liaison_agent,
        "name": "Air Traffic Control Liaison Agent",
        "query": "Predict airspace congestion at LAX for the next 4 hours.",
        "expected_tool_call": "predict_airspace_congestion"
    },
    {
        "agent": disruption_management_agent,
        "name": "Disruption Management Agent",
        "query": "Predict impact for flight AC123 due to severe weather.",
        "expected_tool_call": "predict_disruption_impact"
    },
    {
        "agent": crew_scheduling_agent,
        "name": "Crew Scheduling Agent",
        "query": "Assign pilot P-101 and flight attendants FA-201, FA-202 to flight AC123.",
        "expected_tool_call": "assign_crew_to_flight"
    },
    {
        "agent": aircraft_maintenance_agent,
        "name": "Aircraft Maintenance Agent",
        "query": "Schedule an A-check for aircraft N123UA on 2025-07-15.",
        "expected_tool_call": "schedule_maintenance"
    },
    {
        "agent": operational_data_analysis_agent,
        "name": "Operational Data Analysis Agent",
        "query": "Get telemetry data for flight AC123.",
        "expected_tool_call": "get_flight_data"
    },
    {
        "agent": compliance_safety_agent,
        "name": "Compliance & Safety Agent",
        "query": "Check flight duty limitations for EASA region.",
        "expected_tool_call": "check_aviation_regulation"
    },
    {
        "agent": incident_response_agent,
        "name": "Incident Response Agent",
        "query": "Analyze incident INC-001 using operational data: {\"sensor_readings\": \"engine anomaly detected\", \"flight_phase\": \"takeoff\"}.",
        "expected_tool_call": "analyze_incident"
    },
    {
        "agent": ground_operations_coordination_agent,
        "name": "Ground Operations Coordination Agent",
        "query": "Track baggage loading for flight AC123.",
        "expected_tool_call": "track_ground_activity"
    },
    {
        "agent": passenger_coordinator_agent,
        "name": "Passenger Coordinator Agent",
        "query": "Get passenger manifest for flight AC123.",
        "expected_tool_call": "get_passenger_manifest"
    },
    {
        "agent": passenger_coordinator_agent,
        "name": "Passenger Coordinator Agent",
        "query": "Rebook passenger P001 from AC123 to WS456.",
        "expected_tool_call": "rebook_passenger"
    },
    {
        "agent": crew_coordinator_agent,
        "name": "Crew Coordinator Agent",
        "query": "Check legality for pilot PILOT-001.",
        "expected_tool_call": "check_crew_legality"
    },
    {
        "agent": crew_coordinator_agent,
        "name": "Crew Coordinator Agent",
        "query": "Find available Pilots with Boeing 737 rating.",
        "expected_tool_call": "find_available_crew"
    },
    {
        "agent": flight_dispatcher_agent,
        "name": "Flight Dispatcher Agent",
        "query": "Monitor progress for flight AC123.",
        "expected_tool_call": "monitor_flight_progress"
    },
    {
        "agent": flight_dispatcher_agent,
        "name": "Flight Dispatcher Agent",
        "query": "Analyze weather impact for the route Montreal to Los Angeles for the next 6 hours.",
        "expected_tool_call": "analyze_weather_impact"
    },
    {
        "agent": flight_dispatcher_agent,
        "name": "Flight Dispatcher Agent",
        "query": "Calculate fuel for flight AC123 from Montreal to Los Angeles, payload 15000 kg, 5% contingency.",
        "expected_tool_call": "calculate_fuel_requirements"
    },
    {
        "agent": budget_finance_agent,
        "name": "Budget Finance Agent",
        "query": "Get budget status for Flight Operations for Q3 2025.",
        "expected_tool_call": "get_operational_budget_status"
    },
    {
        "agent": budget_finance_agent,
        "name": "Budget Finance Agent",
        "query": "Log a fuel expense of 15000 for flight AC123.",
        "expected_tool_call": "log_operational_expense"
    },
    {
        "agent": budget_finance_agent,
        "name": "Budget Finance Agent",
        "query": "Forecast fuel costs for the next month.",
        "expected_tool_call": "forecast_operational_costs"
    },
    {
        "agent": budget_finance_agent,
        "name": "Budget Finance Agent",
        "query": "Analyze cost efficiency for recent operations with data: [{\"item\": \"fuel\", \"cost\": 100000}, {\"item\": \"maintenance\", \"cost\": 50000}].",
        "expected_tool_call": "analyze_cost_efficiency"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")
    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f"[DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)


        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f"  Tool Name: {tool_call.function.name}")
                print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f"  [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output}")
                    except json.JSONDecodeError as e:
                        print(f"  [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f"  [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f"  [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call request
                    }
                )
                print(f"  [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n[DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the agent's definition
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)

            # Add the final assistant response to history (optional for a single-turn demo, but good practice)
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")

Mounted at /content/gdrive
Creating AI agents for Airline Operation Control Center Domain...
Flight Planning Agent 'flight-planning-agent' created with ID: ag_0685bf5cd32575a58000b1a45b18963a
Air Traffic Control Liaison Agent 'atc-liaison-agent' created with ID: ag_0685bf5cdc817fc48000b4597c94c137
Disruption Management Agent 'disruption-management-agent' created with ID: ag_0685bf5ce09d73a38000ec299288a2a7
Crew Scheduling Agent 'crew-scheduling-agent' created with ID: ag_0685bf5ce3df7f93800095f493c1155d
Aircraft Maintenance Agent 'aircraft-maintenance-agent' created with ID: ag_0685bf5ce71d762c80004d6836bf1b52
Operational Data Analysis Agent 'operational-data-analysis-agent' created with ID: ag_0685bf5ceab5757980003377e9b4bb6b
Compliance & Safety Agent 'compliance-safety-agent' created with ID: ag_0685bf5cee227049800040c411ccb5ec
Incident Response Agent 'incident-response-agent' created with ID: ag_0685bf5cf1577af980007b48221c4e00
Ground Operations Coordination Agent 'ground-ops-coordi